In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from matplotlib import pyplot as plt
import matplotlib as mpl
from pathlib import Path
from functools import partial

plt.rcParams['figure.figsize'] = [10, 5]


# Definição de diretórios e Disco de leitura dos arquivos (Desktop ou Notebook) 
config_dirs = open("prefixo_dados.txt").readlines() 
DIR_PREFIXO = Path(config_dirs[0].strip())


DIRETORIO_PRINCIPAL =  DIR_PREFIXO / "Projeto_PA" / "Projeto_PA_validado"
DIR_BASE = DIR_PREFIXO / "Projeto_PA" / "dataset_modificado"
DIR_SALVA = DIRETORIO_PRINCIPAL / "analise_abordagens" / "todos_pacientes"


In [2]:
#leitura do dataset completo com dados cadastrais, medidas e estatísticas

dataset = pd.read_csv(DIR_BASE / "pacientes_completos_tratados.csv",sep=';', encoding="Latin1")
    

In [3]:
# PERÍODOS 
#      Matutino 6h-8h59       Dataset 6h-8h45
#      Diurno 9h-20h59                9h-20h45
#      Vespertino 21h-22h59           21h-22h45
#      Noturno 23h-05h59              23h-5h30

#matutino = (time(6,0), time(9,0))
#diurno = (time(9,0), time(21,0))
#vespertino = (time(21,0), time(1,0))
#noturno = (time(1,0), time(6,0))
#
# Valores de Referência Normal das Sistólicas e Diastólicas
max_sist = 140
min_sist = 100
max_diast = 90
min_diast = 60


In [4]:

# Conta a quantidade de Picos = medidas acima do valor de referência

def conta_picos (df, qtd_medidas ,sist_max=140, sist_min=100, diast_max=90, diast_min=60) :
    sist = df.loc[:,'sist 9:00':'sist 8:45']
    diast = df.loc[:,'diast 9:00':'diast 8:45']
    
    saida = pd.DataFrame()
    
    contador = 0  
    
    for medida in range(0, len(sist.columns)):
        if contador == 0 :
            ls_sist_max = list()
            ls_sist_min = list()
            
            if medida >= 56 and medida <= 69 :  
                dia_noite = int(qtd_medidas / 2)
            else :
                dia_noite = qtd_medidas
                
            contador = dia_noite - 1
            colunas = list(sist.iloc[:,medida:medida + dia_noite].columns)
            for linha in sist.iloc[:,medida:medida + dia_noite].values:
                qtd_max = 0
                qtd_min = 0
                for i in range(len(linha)):
                    if linha[i] > sist_max:
                        qtd_max += 1
                    elif linha[i] < sist_min:
                        qtd_min += 1
                ls_sist_max.append(qtd_max)
                ls_sist_min.append(qtd_min)

            nome = 'Sist ' + colunas[0].split(' ')[1] + ' - ' + colunas[-1].split(' ')[1]

            saida[f'{nome} Max'] = ls_sist_max
            saida[f'{nome} Min'] = ls_sist_min
        else : 
            contador -= 1
            
        
    for medida in range(0, len(diast.columns)):
        if contador == 0 :
            ls_diast_max = list()
            ls_diast_min = list()

            if medida >= 56 and medida <= 69 :  
                dia_noite = int(qtd_medidas / 2)
            else :
                dia_noite = qtd_medidas
            contador = dia_noite - 1
            colunas = list(diast.iloc[:,medida:medida + dia_noite].columns)
            for linha in diast.iloc[:,medida:medida + dia_noite].values:
                qtd_max = 0
                qtd_min = 0
                for i in range(len(linha)):
                    if linha[i] > diast_max:
                        qtd_max += 1
                    elif linha[i] < diast_min:
                        qtd_min += 1
                ls_diast_max.append(qtd_max)
                ls_diast_min.append(qtd_min)

            nome = 'Diast ' + colunas[0].split(' ')[1] + ' - ' + colunas[-1].split(' ')[1]

            saida[f'{nome} Max'] = ls_diast_max
            saida[f'{nome} Min'] = ls_diast_min
        else : 
            contador -= 1
        
    return saida
 

In [5]:
#  Calcula a qtd de picos e organiza em forma descendente
# Informa à função o dataset e quantas medidas considerar. A cada 4, significa 1h de medidas diurnas

contagem_picos = conta_picos(dataset,4)
ls_valores = list()
for i in range(len(contagem_picos)) :
    ls_valores.append(sum(contagem_picos.iloc[i,:]))
novo_dataset = pd.DataFrame ()
novo_dataset['numero_identificacao'] = dataset['numero_identificacao']
novo_dataset ['qtd picos'] = ls_valores
novo_dataset = novo_dataset.sort_values(by=['qtd picos'], ascending=False)


In [6]:
#  Grava a quantidade de picos por cada período de tempo informado para calcular
qtd_total_picos = np.sum(contagem_picos,axis=0).sort_values(ascending=False)
qtd_total_picos.to_csv ( DIR_SALVA / "frequencia_picos.csv", sep=";", encoding="latin1", header=False)